In [1]:
import os
# если я правильно помню, без такой переменной gym будет пытаться отрендерить изображение на гпу и могут возникнуть проблемы на кластере.
os.environ["SDL_VIDEODRIVER"] = "dummy"
os.chdir("..")  # !!!  сменим папке, чтоб сервер ray увидел код continuous_grid_arctic (возможно как-то подругому можно, sys.path не помогает)

In [2]:
import gym
import pandas as pd
import ray
from ray.rllib.agents.ppo.ppo import PPOTrainer
#import pygame
#import matplotlib.pyplot as plt
#%matplotlib inline

import continuous_grid_arctic.follow_the_leader_continuous_env  # при импорте происходит регистрация сред
from continuous_grid_arctic.utils.wrappers import ContinuousObserveModifier_v0
from ray.tune import register_env

pygame 2.1.2 (SDL 2.0.16, Python 3.8.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
def continuous_env_maker(config):
    """
    Функция создающая среду для робота по переданному конфигу.
    В конфиге должны быть поля:
     - name - имя среды, под которым она была зарегистрирована через gym_register. # сейчас регистрация происходит при импорте follow_the_leader_continuous_env
     - base_env_config - словарь с аргументами, которые будут переданы в конструктор класса gym среды
     - wrappers - список названий классов обёрток, которые будут добавлены к среде для обработки наблюдений и экшенов.
    """
    name = config["name"]
    action_values_range = config.get("action_values_range", None)
    env = gym.make(name, **config["base_env_config"])
    if 'ContinuousObserveModifier_v0' in config["wrappers"]:
        env = ContinuousObserveModifier_v0(env, action_values_range)    
    return env

register_env("continuous-grid", continuous_env_maker)

In [4]:
env_config = {
    'name' : 'Test-Cont-Env-Auto-v0',  #  имя среды, под которым она была зарегистрирована через gym_register. # сейчас регистрация происходит при импорте follow_the_leader_continuous_env, посмотреть доступные названия можно в follow_the_leader_continuous_env.py
    'wrappers': ['ContinuousObserveModifier_v0'],  # список названий классов обёрток, которые будут добавлены к среде для обработки наблюдений и экшенов.
    'base_env_config' : {  # словарь с аргументами, которые будут переданы в конструктор класса gym среды
        'add_obstacles': True,  # наличие препятствий в среде
        'max_distance': 4,  # максимальная дистанция, на которую может отстать ведомый (метры)
        'warm_start': 0,  # количество шагов, в течение которых не начислаются награды и штрафы
        'max_steps': 10000,  # количество шагов, после которого остановится симуляция
        'framerate': 5000,  # ? максимальная частота шагов?
        'obstacle_number': 35,  # количество камней
        'constant_follower_speed': False,  # зафиксировать скорость ведомого и оставить только 1 действие - поворот
        'corridor_length': 10,  # длина коридора сейф зоны (с точки зрения признака, но не награды)  # !!! вообще это должен быть параметр сенсора
        'corridor_width': 1.5,  # ширина коридора сейф зоны (с точки зрения признака, но не награды)  # !!! вообще это должен быть параметр сенсора
        'move_bear_v4': True,  #?
        'follower_speed_koeff': 0.6,  # коэффициент скорости ведомого
        'leader_speed_coeff': 0.45,  # коэффициент скорости лидера
        'negative_speed': True,  # можно ли двигаться назад
        'add_bear': True,  # добавлять ли медведя (динамическое препатствие)
        'bear_number': 1,  # количество динамических препятствий
        'bear_speed_coeff': 1.2,  # коэффициент скорости медведя                
        'multi_random_bears': False,  # ?
        'bear_behind': False,  # ?
        'use_prev_obs': True,  # использовать предыдущие наблюдения
        'max_prev_obs': 5,  # количество предыдущих наблюдений
        'early_stopping' : {  # когда стоит останавливать симуляцию (помимо максимального количества шагов)
            'max_distance_coef': 3.5,  # когда ведомый удалится от лидера больше чем на max_distance*max_distance_coef метров
            'low_reward': -300  # когда суммарная награда опустится ниже этого значения
        },
        'random_frames_per_step': [2, 20],  # количество фреймов (изменений происходящих в среде) на один шаг (принятие решения)
        'leader_speed_regime': {  # расписание коэффициентов скорости лидера
            '0': [0.2, 1],
            '200': 1,
            '1000': [0.5, 1],
            '1500': 0.75,
            '2000': 0,
            '2500': 1,
            '3000': [0.5, 1],
            '4000': [0.0, 0.5],
            '5000': [0.4, 1]
        },
        'follower_sensors': {  # активные сенсоры ведомого и их настройки
            'LeaderPositionsTracker_v2': {  
                'eat_close_points': False,
                'generate_corridor': True,
                'saving_period': 8,
                'sensor_name': 'LeaderPositionsTracker_v2',
                'start_corridor_behind_follower': True,
            },
            'LeaderCorridor_Prev_lasers_v2': {
                'sensor_name': 'LeaderCorridor_Prev_lasers_v2',
                'react_to_obstacles':  True,
                'react_to_safe_corridor': True,
                'react_to_green_zone': True,
                'front_lasers_count': 6,
                'back_lasers_count': 6,
                'laser_length': 150
            },
            'LaserPrevSensor': {
                'sensor_name': 'LaserPrevSensor',
                'react_to_obstacles': True,
                'react_to_safe_corridor': False,
                'react_to_green_zone': False,
                'front_lasers_count': 15,
                'back_lasers_count':15,
                'laser_length': 200
            }
        },
    }
    
}

run_config = { 
    'env': 'continuous-grid',
    'run': 'PPO',
    'local_dir': 'notebooks/demo_run_logs',
    'checkpoint_freq': 20,
    'stop': {
        'training_iteration': 100,
    },
    'config': {
        'num_gpus': 1,
        #'timesteps_per_iteration': 1000,
        'num_workers': 1,
        'log_level': 'ERROR',
        'framework': 'tf',
        'env': 'continuous-grid',
        'normalize_actions': False,
        'env_config': env_config,
        'train_batch_size': 10000,
        'sgd_minibatch_size': 256,
       # 'model': {
            #'custom_model': 'def_m_prev',
       # }
    }
}
        

In [5]:
# конфигурация запуска рэя
# https://docs.ray.io/en/releases-1.11.0/rllib/rllib-training.html#common-parameters
# https://docs.ray.io/en/releases-1.11.0/rllib/rllib-algorithms.html#proximal-policy-optimization-ppo
run_config = {
    'num_gpus': 1,
    'num_workers': 1,
    'log_level': 'ERROR',
    'framework': 'tf',
    'env': 'continuous-grid',
    'normalize_actions': False,
    'env_config': env_config,
    'train_batch_size': 1024,  # сколько шагов будет сэмплировано перед тем, как начать обучение. 
    'sgd_minibatch_size': 256
}        

In [6]:
ray.init()

{'node_ip_address': '172.19.8.129',
 'raylet_ip_address': '172.19.8.129',
 'redis_address': '172.19.8.129:6379',
 'object_store_address': '/tmp/ray/session_2023-06-09_17-27-53_928869_24893/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2023-06-09_17-27-53_928869_24893/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2023-06-09_17-27-53_928869_24893',
 'metrics_export_port': 59216,
 'node_id': '99de16e6adf264f5747dd86dc417df96d590049d2167756caaceeafc'}

# Train

In [7]:
from ray.rllib.agents.ppo import PPOTrainer
rllib_trainer = PPOTrainer(config=run_config)
print(rllib_trainer.logdir)

2023-06-09 17:27:57,095	INFO trainer.py:722 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also want to then set `eager_tracing=True` in order to reach similar execution speed as with static-graph mode.
2023-06-09 17:27:57,096	WARNING ppo.py:143 -- `train_batch_size` (1024) cannot be achieved with your other settings (num_workers=1 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 1024.
2023-06-09 17:27:57,097	INFO ppo.py:166 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2023-06-09 17:27:57,098	INFO trainer.py:743 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=25151) pygame 2.1.2 (SDL 2.0.16, Python 3.8.0)
(RolloutWorker pid=25151) Hello from the pygame community. https://www.pygame.org/contribute.html
(RolloutWorker pid=25151) ===Запуск симуляции номер 0===


(RolloutWorker pid=25151) /s/ls4/users/grartem/RL_robots/continuous_grid_arctic/continuous_grid_arctic/follow_the_leader_continuous_env.py:331: UserWarning: Одновременно заданы и random_frames_per_step и frames_per_step, будет использоваться random_frames_per_step
(RolloutWorker pid=25151)   warn(
(RolloutWorker pid=25151) /s/ls4/users/grartem/anaconda3/envs/rl_robots/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=25151)   logger.warn(
(RolloutWorker pid=25151) Failed to create secure directory (/run/user/1556/pulse): No such file or directory
(RolloutWorker pid=25151) ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
(RolloutWorker pid=25151) ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
(RolloutWorker pid=25151) ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
(RolloutWorker pid=25151) ALSA lib conf.

/s/ls4/users/grartem/ray_results/PPO_continuous-grid_2023-06-09_17-27-57jopp9fq3/


In [ ]:
train_results = []
for iter_i in range(5):
    print("\n### ITERATION:\n", iter_i)
    # запускаем сбор данных + обучение
    iter_results = rllib_trainer.train()
    # сохраняем чекпоинт
    checkpoint_path = rllib_trainer.save(rllib_trainer.logdir)
    iter_results.pop("config")
    train_results.append(iter_results)

In [9]:
import pandas  as pd
pd.DataFrame(train_results)

,episode_reward_max,episode_reward_min,episode_reward_mean,episode_len_mean,episode_media,episodes_this_iter,policy_reward_min,policy_reward_max,policy_reward_mean,custom_metrics,...,timestamp,time_this_iter_s,time_total_s,pid,hostname,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,perf
0,44.7,-5.5,15.557143,142.00000,{},7,{},{},{},{},...,1686320970,78.085608,78.085608,24893,g001,172.19.8.129,78.085608,0,1,"{'cpu_util_percent': 3.7017857142857147, 'ram_..."
1,81.5,-14.0,19.800000,143.00000,{},7,{},{},{},{},...,1686321044,73.335085,151.420692,24893,g001,172.19.8.129,151.420692,0,2,"{'cpu_util_percent': 3.697142857142857, 'ram_u..."
2,81.5,-14.0,18.386957,128.26087,{},9,{},{},{},{},...,1686321126,81.775246,233.195938,24893,g001,172.19.8.129,233.195938,0,3,"{'cpu_util_percent': 3.6724137931034484, 'ram_..."
3,81.5,-14.0,27.118750,126.62500,{},9,{},{},{},{},...,1686321205,79.407805,312.603744,24893,g001,172.19.8.129,312.603744,0,4,"{'cpu_util_percent': 3.7166666666666672, 'ram_..."
4,81.5,-14.0,25.661905,120.50000,{},10,{},{},{},{},...,1686321302,96.838297,409.442040,24893,g001,172.19.8.129,409.442040,0,5,"{'cpu_util_percent': 3.6978260869565216, 'ram_..."


# Predict

In [10]:
# Загружаем последний чекпоинт
rllib_trainer.restore(checkpoint_path)

2023-06-09 17:35:03,332	INFO trainable.py:467 -- Restored on 172.19.8.129 from checkpoint: /s/ls4/users/grartem/ray_results/PPO_continuous-grid_2023-06-09_17-27-57jopp9fq3/checkpoint_000005/checkpoint-5
2023-06-09 17:35:03,334	INFO trainable.py:475 -- Current state after restoring: {'_iteration': 5, '_timesteps_total': 0, '_time_total': 409.44204020500183, '_episodes_total': 42}


In [13]:
# создаём экземпляр среды для теста (возомжно его как-то из тренера можно достать)
env = continuous_env_maker(config=env_config)
# добавляем обёртку из gym для сохранения видео
monitor_env = gym.wrappers.Monitor(env=env,
                                    directory= os.path.dirname(checkpoint_path)+"/videos",
                                   video_callable=lambda _:True,
                                   force=False,
                                  uid="video_test", mode="evaluation")

monitor_env.seed(12)  # выбираем какой-нибудь рандом сид (фиксируем расположение объектов)
obs = monitor_env.reset()  # сбрасываем среду  (сид применяется как раз здесь), получаем первое наблюдение
done = False
while not done:
    # определяем действие по наблюдению
    action = rllib_trainer.compute_single_action(obs, explore=False)
    # применяем действие, получаем следующее наблюдение и награду
    obs, reward, done, info = monitor_env.step(action)
obs = monitor_env.reset()  # чтобы завершить видео файл, надо вызвать сброс. 

===Запуск симуляции номер 2===
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
===Запуск симуляции номер 3===
